In [52]:
import pandas as pd
import random
import urllib3
import os

In [53]:
def download(url, path):
    http = urllib3.PoolManager()
    r = http.request('GET', url, preload_content=False)

    with open(path, 'wb') as out:
        while True:
            data = r.read(48000)
            if not data:
                break
            out.write(data)

    r.release_conn()

In [54]:
labels = [
    'airplane',
    'car', 
    'horse'
]
labels_ids = [
    '/m/0cmf2', 
    '/m/0k4j', 
    '/m/03k3r'
]
n = 100

In [55]:
validation = pd.read_csv("/Users/racoon/Desktop/open-images-v5-metadata/validation-annotations-bbox.csv")

In [56]:
subsets = [validation[(validation['LabelName']==labels_id) & (validation["Confidence"] == 1) & (validation["IsDepiction"] == 0) & (validation["IsGroupOf"] == 0)] for labels_id in labels_ids]
other_rows = validation[(~validation['LabelName'].isin(labels_ids)) & (validation["Confidence"] == 1) & (validation["IsDepiction"] == 0) & (validation["IsGroupOf"] == 0)]

In [57]:
candidates = [[] for _ in range(len(labels)+1)]
u = set()

for i in range(len(subsets)):
    rows = subsets[i]
    for index, row in rows.iterrows():
        area = (row["XMax"]-row["XMin"]) * (row["YMax"]-row["YMin"])
        if area >= 0.2:
            u.add(row["ImageID"])
            candidates[i].append(row["ImageID"])

while len(candidates[-1]) < n:        
    row = other_rows.sample().iloc[0]

    if row["ImageID"] not in u:
        candidates[-1].append(row["ImageID"])

In [58]:
members = [random.sample(c, n) for c in candidates]

In [ ]:
for i, m in enumerate(members):
    name = labels[i] if i < len(subsets) else 'other'
    os.makedirs(f'dataset_{n}/{name}', exist_ok=True)
    for id in m:
        print(f'http://s3.amazonaws.com/open-images-dataset/validation/{id}.jpg', f'dataset_{n}/{name}/{id}.jpg')
        download(f'http://s3.amazonaws.com/open-images-dataset/validation/{id}.jpg', f'dataset_{n}/{name}/{id}.jpg')

http://s3.amazonaws.com/open-images-dataset/validation/615f8f65c561d00e.jpg dataset_100/airplane/615f8f65c561d00e.jpg
http://s3.amazonaws.com/open-images-dataset/validation/3faa503009c70368.jpg dataset_100/airplane/3faa503009c70368.jpg
http://s3.amazonaws.com/open-images-dataset/validation/8e43b7eaf13339a2.jpg dataset_100/airplane/8e43b7eaf13339a2.jpg
http://s3.amazonaws.com/open-images-dataset/validation/43dc96250edb4dca.jpg dataset_100/airplane/43dc96250edb4dca.jpg
http://s3.amazonaws.com/open-images-dataset/validation/de16d7208ce4420a.jpg dataset_100/airplane/de16d7208ce4420a.jpg
http://s3.amazonaws.com/open-images-dataset/validation/9261dd04cd85adb5.jpg dataset_100/airplane/9261dd04cd85adb5.jpg
http://s3.amazonaws.com/open-images-dataset/validation/25a12c41908eee85.jpg dataset_100/airplane/25a12c41908eee85.jpg
http://s3.amazonaws.com/open-images-dataset/validation/ae3322cdf2feb016.jpg dataset_100/airplane/ae3322cdf2feb016.jpg
http://s3.amazonaws.com/open-images-dataset/validation/c